## Librairies used 

In [1]:
import requests
import json
import gzip
import shutil
import time
import os
from io import BytesIO
import pandas as pd
import datetime as dt
import random

## Utilities function for data preparation 

In [5]:
def extract_game_state_data(game_event,value=0) :
    '''We process the relevant status updates and add them to dictionaries to output.
    The game event and the list of participants on both teams must be provided.
    By default, without any parameter, the function will treat this event as an endgame
    event depending on the game status. Please specify either 10 or 15 on 'value' for those
    two events.'''
    dict_output = {}
    gameOver = (game_event.get('eventType') == 'game_state_end')
    value = 'End' if gameOver else value 

    player_data = [i for i in game_event.get('participants') if (0 < i.get('participantID') < 11)]
    player_data = sorted(player_data, key = lambda x: x['participantID'])

    if (value == 10 or value == 15) or gameOver:

        #Initializing variables:
        gold_top_blue = player_data[0]['totalGold']
        gold_top_red = player_data[5]['totalGold']
        gold_diff_top = gold_top_blue - gold_top_red
        
        gold_jg_blue = player_data[1]['totalGold']
        gold_jg_red = player_data[6]['totalGold']
        gold_diff_jg = gold_jg_blue - gold_jg_red
        
        gold_mid_blue = player_data[2]['totalGold']
        gold_mid_red = player_data[7]['totalGold']
        gold_diff_mid = gold_mid_blue - gold_mid_red
        
        gold_ad_blue = player_data[3]['totalGold']
        gold_ad_red = player_data[8]['totalGold']
        gold_diff_ad = gold_ad_blue - gold_ad_red
        
        gold_sup_blue = player_data[4]['totalGold']
        gold_sup_red = player_data[9]['totalGold']
        gold_diff_sup = gold_sup_blue - gold_sup_red
        
        xp_diff_top = player_data[0]['XP'] - player_data[5]['XP']
        xp_diff_jg = player_data[1]['XP'] - player_data[6]['XP']
        xp_diff_mid = player_data[2]['XP'] - player_data[7]['XP']
        xp_diff_ad = player_data[3]['XP'] - player_data[8]['XP']
        xp_diff_sup = player_data[4]['XP'] - player_data[9]['XP']
        dict_output.update({
            f'GoldTopBlue{value}': gold_top_blue,
            f'GoldJgBlue{value}': gold_jg_blue,
            f'GoldMidBlue{value}': gold_mid_blue,
            f'GoldADBlue{value}': gold_ad_blue,
            f'GoldSupBlue{value}': gold_sup_blue,
            f'GoldTopRed{value}': gold_top_red,
            f'GoldJgRed{value}': gold_jg_red,
            f'GoldMidRed{value}': gold_mid_red,
            f'GoldADRed{value}': gold_ad_red,
            f'GoldSupRed{value}': gold_sup_red,
            f'GoldDiff{value}Top': gold_diff_top,
            f'GoldDiff{value}Jg': gold_diff_jg,
            f'GoldDiff{value}Mid': gold_diff_mid,
            f'GoldDiff{value}AD': gold_diff_ad,
            f'GoldDiff{value}Bot': gold_diff_sup,
            f'XPDiff{value}Top': xp_diff_top,
            f'XPDiff{value}Jg': xp_diff_jg,
            f'XPDiff{value}Mid': xp_diff_mid,
            f'XPDiff{value}AD': xp_diff_ad,
            f'XPDiff{value}Bot': xp_diff_sup,
        })
        
        blue_kills, red_kills = game_event['blue']['championsKills'], game_event['red']['championsKills']
        blue_assists, red_assists = game_event['blue']['assists'], game_event['red']['assists']
        blue_deaths, red_deaths = game_event['blue']['deaths'], game_event['red']['deaths']
        blue_gold, red_gold = game_event['blue']['totalGold'], game_event['red']['totalGold']
        blue_tower_kills, red_tower_kills = game_event['blue']['towerKills'], game_event['red']['towerKills']
        blue_dragon_kills, red_dragon_kills = game_event['blue']['dragonKills'], game_event['red']['dragonKills']
        dict_output.update({
            f'BlueKills{value}': blue_kills,
            f'BlueAssists{value}': blue_assists,
            f'BlueDeaths{value}': blue_deaths,
            f'BlueTotalGold{value}': blue_gold,
            f'BlueDragonKills{value}': blue_dragon_kills,
            f'BlueTowerKills{value}': blue_tower_kills,
            f'RedKills{value}': red_kills,
            f'RedAssists{value}': red_assists,
            f'RedDeaths{value}': red_deaths,
            f'RedTotalGold{value}': red_gold,
            f'RedDragonKills{value}': red_dragon_kills,
            f'RedTowerKills{value}': red_tower_kills
        })
        
        if value != 10:
            blue_inhib_kills, red_inhib_kills = game_event['blue']['inhibKills'], game_event['red']['inhibKills']
            dict_output.update({
                f'BlueInhibKills{value}': blue_inhib_kills,
                f'RedInhibKills{value}': red_inhib_kills
            })
            
        if gameOver:
            blue_baron_kills, red_baron_kills = game_event['blue']['baronKills'], game_event['red']['baronKills']
            dict_output.update({
                f'BlueBaronKills{value}': blue_baron_kills,
                f'RedBaronKills{value}': red_baron_kills
            })
            
            blue_vision_score_top = player_data[0]['VISION_SCORE']
            blue_vision_score_jg = player_data[1]['VISION_SCORE']
            blue_vision_score_mid = player_data[2]['VISION_SCORE']
            blue_vision_score_ad = player_data[3]['VISION_SCORE']
            blue_vision_score_sup = player_data[4]['VISION_SCORE']
            red_vision_score_top = player_data[5]['VISION_SCORE']
            red_vision_score_jg = player_data[6]['VISION_SCORE']
            red_vision_score_mid = player_data[7]['VISION_SCORE']
            red_vision_score_ad = player_data[8]['VISION_SCORE']
            red_vision_score_sup = player_data[9]['VISION_SCORE']

            dict_output.update({
                'VisionScoreTopBlue': blue_vision_score_top,
                'VisionScoreJgBlue': blue_vision_score_jg,
                'VisionScoreMidBlue': blue_vision_score_mid,
                'VisionScoreADBlue': blue_vision_score_ad,
                'VisionScoreSupBlue': blue_vision_score_sup,
                'VisionScoreTopRed': red_vision_score_top,
                'VisionScoreJgRed': red_vision_score_jg,
                'VisionScoreMidRed': red_vision_score_mid,
                'VisionScoreADRed': red_vision_score_ad,
                'VisionScoreSupRed': red_vision_score_sup
            })

            blue_damage_dealt_top = player_data[0]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            blue_damage_dealt_jg = player_data[1]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            blue_damage_dealt_mid = player_data[2]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            blue_damage_dealt_ad = player_data[3]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            blue_damage_dealt_sup = player_data[4]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            red_damage_dealt_top = player_data[5]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            red_damage_dealt_jg = player_data[6]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            red_damage_dealt_mid = player_data[7]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            red_damage_dealt_ad = player_data[8]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']
            red_damage_dealt_sup = player_data[9]['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS']

            dict_output.update({
                'DamageDealtTopBlue': blue_damage_dealt_top,
                'DamageDealtJgBlue': blue_damage_dealt_jg,
                'DamageDealtMidBlue': blue_damage_dealt_mid,
                'DamageDealtADBlue': blue_damage_dealt_ad,
                'DamageDealtSupBlue': blue_damage_dealt_sup,
                'DamageDealtTopRed': red_damage_dealt_top,
                'DamageDealtJgRed': red_damage_dealt_jg,
                'DamageDealtMidRed': red_damage_dealt_mid,
                'DamageDealtADRed': red_damage_dealt_ad,
                'DamageDealtSupRed': red_damage_dealt_sup
            })
            
            blue_damage_taken_top = player_data[0]['TOTAL_DAMAGE_TAKEN']
            blue_damage_taken_jg = player_data[1]['TOTAL_DAMAGE_TAKEN']
            blue_damage_taken_mid = player_data[2]['TOTAL_DAMAGE_TAKEN']
            blue_damage_taken_ad = player_data[3]['TOTAL_DAMAGE_TAKEN']
            blue_damage_taken_sup = player_data[4]['TOTAL_DAMAGE_TAKEN']
            red_damage_taken_top = player_data[5]['TOTAL_DAMAGE_TAKEN']
            red_damage_taken_jg = player_data[6]['TOTAL_DAMAGE_TAKEN']
            red_damage_taken_mid = player_data[7]['TOTAL_DAMAGE_TAKEN']
            red_damage_taken_ad = player_data[8]['TOTAL_DAMAGE_TAKEN']
            red_damage_taken_sup = player_data[9]['TOTAL_DAMAGE_TAKEN']

            dict_output.update({
                'DamageTakenTopBlue': blue_damage_taken_top,
                'DamageTakenJgBlue': blue_damage_taken_jg,
                'DamageTakenMidBlue': blue_damage_taken_mid,
                'DamageTakenADBlue': blue_damage_taken_ad,
                'DamageTakenSupBlue': blue_damage_taken_sup,
                'DamageTakenTopRed': red_damage_taken_top,
                'DamageTakenJgRed': red_damage_taken_jg,
                'DamageTakenMidRed': red_damage_taken_mid,
                'DamageTakenADRed': red_damage_taken_ad,
                'DamageTakenSupRed': red_damage_taken_sup
            })

            blue_cc_score_top = player_data[0]['TIME_CCING_OTHERS']
            blue_cc_score_jg = player_data[1]['TIME_CCING_OTHERS']
            blue_cc_score_mid = player_data[2]['TIME_CCING_OTHERS']
            blue_cc_score_ad = player_data[3]['TIME_CCING_OTHERS']
            blue_cc_score_sup = player_data[4]['TIME_CCING_OTHERS']
            red_cc_score_top = player_data[5]['TIME_CCING_OTHERS']
            red_cc_score_jg = player_data[6]['TIME_CCING_OTHERS']
            red_cc_score_mid = player_data[7]['TIME_CCING_OTHERS']
            red_cc_score_ad = player_data[8]['TIME_CCING_OTHERS']
            red_cc_score_sup = player_data[9]['TIME_CCING_OTHERS']

            dict_output.update({
                'TotalCCDurationTopBlue': blue_cc_score_top,
                'TotalCCDurationJgBlue': blue_cc_score_jg,
                'TotalCCDurationMidBlue': blue_cc_score_mid,
                'TotalCCDurationADBlue': blue_cc_score_ad,
                'TotalCCDurationSupBlue': blue_cc_score_sup,
                'TotalCCDurationTopRed': red_cc_score_top,
                'TotalCCDurationJgRed': red_cc_score_jg,
                'TotalCCDurationMidRed': red_cc_score_mid,
                'TotalCCDurationADRed': red_cc_score_ad,
                'TotalCCDurationSupRed': red_cc_score_sup
            })            

    return dict_output

In [6]:
def add_event_to_counter(event_counter, player_index, team, team_ids_blue, team_ids_red):
    '''We get an event and assign it to the correct team.
    The code takes under account possible lapses in our data (if we don't have a 'team'
    but have the list of participants).'''
    
    changed_counter = event_counter
    team_result = team
    
    if team_result is None:
        if player_index in team_ids_blue:
            team_result = 'blue'
        elif player_index in team_ids_red:
            team_result = 'red'

    #This is not an ELSE situation. I want to run this after we attempt to fix the issue above.
    if team_result is not None:
        changed_counter[team_result.lower()]+=1

    return (changed_counter, team_result)

In [7]:
def final_data(game_json):
    dict_stats = {}
    game_info_dict = {}
    game_state_10 = {}
    game_state_15 = {}
    game_state_end = {}
    header_stats = None
    wards_placed = [0 for i in range(0,10)]
    control_wards_placed = [0 for i in range(0,10)]
    wards_killed = [0 for i in range(0,10)]
    control_wards_killed = [0 for i in range(0,10)]
    current_event = None
    previous_event = None
    game_duration = 7200 #Put it at two hours for funsies.
    game_winner = None
    dragon_type_queued = [None, None]
    dragon_index = 0
    dragon_soul_taken = False
    dragon_soul = {
        'DragonSoulTimer':None,
        'DragonSoulType':None,
        'DragonSoulTaker':None
    }

    default_dict = {'blue':0,'red':0}
    elder_dragons_taken = {'blue':0,'red':0}
    own_camps_taken = {'blue':0,'red':0}
    enemy_camps_taken = {'blue':0,'red':0}
    rift_herald_count = {'blue':0,'red':0}
    scuttle_count = {'blue':0,'red':0}
    dragon_count = {'blue':0,'red':0}
    dragon_type_count = {'blue':{}, 'red':{}}
    baron_count = {'blue':0,'red':0}
    tower_count = {'blue':0,'red':0}
    tower_log = {
        'OuterTopBlueTimer': None,
        'OuterMidBlueTimer': None,
        'OuterBotBlueTimer': None,
        'InnerTopBlueTimer': None,
        'InnerMidBlueTimer': None,
        'InnerBotBlueTimer': None,
        'BaseTopBlueTimer': None,
        'BaseMidBlueTimer': None,
        'BaseBotBlueTimer': None,
        'Nexus1MidBlueTimer': None,
        'Nexus2MidBlueTimer': None,
        'OuterTopRedTimer': None,
        'OuterMidRedTimer': None,
        'OuterBotRedTimer': None,
        'InnerTopRedTimer': None,
        'InnerMidRedTimer': None,
        'InnerBotRedTimer': None,
        'BaseTopRedTimer': None,
        'BaseMidRedTimer': None,
        'BaseBotRedTimer': None,
        'Nexus1MidRedTimer': None,
        'Nexus2MidRedTimer': None
    } #Time values for each event
    team_side = None

    smaller_camps = ['blueCamp','redCamp','gromp','wolf','krug','raptor']

    #Data integrity check, but also getting relevant data.
    if game_json[-1].get('eventType',None) != 'game_end':
        print("The game's data does not contain the entirety of the game.")
        #TO_DO: plug Tim's game_end timestamp (game duration)
    else:
        game_duration = game_json[-1].get('gameTime')
        game_winner = game_json[-1].get('winningTeam')
        
    for game_event in game_json:
        
        # In case we have our intro header somewhere in there:
        if game_event.get('eventType',None) is None:
            
            try:
                game_date_str = game_event.get('gameDate', None)
                if game_date_str:
                    game_date_str = game_date_str.replace('Z', '+00:00')  # Replace 'Z' with '+00:00'
                    game_date = dt.datetime.fromisoformat(game_date_str)
            except ValueError as e:
                print(f"Error parsing date: {e}")

            esports_platform = game_event.get('esportsPlatformId',None)
            game_patch = game_event.get('gameVersion',None)
            header_stats = {
                'gameDate': game_date,
                'esportsPlatformId': esports_platform,
                'gameVersion': game_patch
            }

        else:
            current_event = game_event.get('eventType',None)

        # Important bit: we get our game info.
        if game_event.get('eventType') == 'game_info':
            blue_participant_id = []
            red_participant_id = []
            blue_summoner_name = []
            red_summoner_name = []
            blue_champion = []
            red_champion = []
            
            blue_data = game_event.get('blue')
            for player in blue_data:
                blue_participant_id.append(player.get('participantID'))
                blue_summoner_name.append(player.get('summonerName'))
                blue_champion.append(player.get('championName'))

            red_data = game_event.get('red')
            for player in red_data:
                red_participant_id.append(player.get('participantID'))
                red_summoner_name.append(player.get('summonerName'))
                red_champion.append(player.get('championName'))

        if game_event.get('eventType') == 'queued_dragon_info':
            dragon_index = dragon_index + 1
            if dragon_type_queued[0] is None:
                dragon_type_queued[0] = game_event.get('nextDragonName')
            else:
                dragon_type_queued[1] = game_event.get('nextDragonName')

        if game_event.get('eventType') == 'epic_monster_kill':
            #So, there are A LOT of epic monsters killed.
            #The events have the "killer" in common.
            killer = game_event.get('killer')
            killer_team = game_event.get('team')
            
            #Step 1: counterjungle tally
            if game_event.get('inEnemyJungle'):
                #Temporary workaround given data mistakes were made
                enemy_camps_taken = add_event_to_counter(enemy_camps_taken, killer, killer_team,
                                                        blue_participant_id, red_participant_id)[0]
            #If no counterjungling is afoot, take this number instead:
            elif game_event.get('monsterType') in smaller_camps:
                own_camps_taken = add_event_to_counter(own_camps_taken, killer, killer_team,
                                                        blue_participant_id, red_participant_id)[0]

            #Step 2: Scuttles, Heralds, Barons, and Dragons
            if game_event.get('monsterType') == 'scuttleCrab':
                scuttle_count = add_event_to_counter(scuttle_count, killer, killer_team,
                                                        blue_participant_id, red_participant_id)[0]

            if game_event.get('monsterType') == 'riftHerald':
                rift_herald_count = add_event_to_counter(rift_herald_count, killer, killer_team,
                                                        blue_participant_id, red_participant_id)[0]

            if game_event.get('monsterType') == 'baron':
                baron_count = add_event_to_counter(baron_count, killer, killer_team,
                                                        blue_participant_id, red_participant_id)[0]

            if game_event.get('monsterType') == 'dragon':
                dragon_count, team_side = add_event_to_counter(dragon_count, killer, killer_team,
                                                        blue_participant_id, red_participant_id)

                if team_side is not None:
                    if not dragon_soul_taken:
                        nb_dragons = dragon_count[team_side]
                        if nb_dragons == 4:
                            dragon_soul_taken = True
                            dragon_soul = {
                                'DragonSoulTimer':game_event.get('gameTime'),
                                'DragonSoulType':dragon_type_queued[0],
                                'DragonSoulTeam':team_side
                            }
                    else:
                        elder_dragons_taken = add_event_to_counter(elder_dragons_taken, killer, killer_team,
                                                        blue_participant_id, red_participant_id)[0]
                            

                    
                if dragon_type_queued[1] is not None:
                    dragon_type_queued[0] = dragon_type_queued[1]
                    dragon_type_queued[1] = None

        if game_event.get('eventType') == 'building_destroyed':
            try:
                team_side = game_event.get('team',None)
                building_type = game_event.get('buildingType')
                
                if building_type == 'turret' and team_side is not None:
                    building_lane = game_event.get('lane').title()
                    building_tier = game_event.get('turretTier').title()
                    
                    if building_tier == 'Nexus':
                        if tower_log.get(f'{building_tier}1{building_lane}{team_side.title()}Timer',None) is not None:
                            building_tier = 'Nexus2'
                        else:
                            building_tier = 'Nexus1'
                            
                    tower_log.update({f'{building_tier}{building_lane}{team_side.title()}Timer':
                                    game_event.get('gameTime')})
                    tower_count = add_event_to_counter(tower_count, None, team_side,
                                                        None, None)[0]
            except:
                print('Did a building self-destruct by any chance? Something has gone awry here.')

        if game_event.get('eventType') == 'game_state_10mn':
            game_state_10 = extract_game_state_data(game_event,10)

        if game_event.get('eventType') == 'game_state_15mn':
            game_state_15 = extract_game_state_data(game_event,15)
        
        if game_event.get('eventType') == 'game_state_end':
            game_state_end = extract_game_state_data(game_event)
            
        previous_event = current_event
        
            
    #This will be our first entry in the dictionaries.
    if header_stats is not None:
        dict_stats.update(header_stats)

    dict_stats.update({
        'NbCampsSecuredBlue': own_camps_taken['blue'],
        'NbCampsSecuredRed': own_camps_taken['red'],
        'NbCampsStolenBlue': enemy_camps_taken['blue'],
        'NbCampsStolenRed': enemy_camps_taken['red'],
        'NbScuttlesBlue': scuttle_count['blue'],
        'NbScuttlesRed': scuttle_count['red'],
        'NbRiftHeraldsBlue': rift_herald_count['blue'],
        'NbRiftHeraldsRed': rift_herald_count['red'],
        'NbDragonsBlue': dragon_count['blue']-elder_dragons_taken['blue'], #Denotes only regular dragons
        'NbDragonsRed': dragon_count['red']-elder_dragons_taken['red'],
        'NbBaronsBlue': baron_count['blue'],
        'NbBaronsRed': baron_count['red'],
        'NbEldersBlue': elder_dragons_taken['blue'],
        'NbEldersRed': elder_dragons_taken['red'],
        'NbTowersBlue': tower_count['red'],
        'NbTowersRed': tower_count['blue']
    })

    dict_stats.update(tower_log)
    dict_stats.update(dragon_soul)
    dict_stats.update(game_state_10)
    dict_stats.update(game_state_15)
    dict_stats.update(game_state_end)
    dict_stats.update({'winner': game_winner})

    return dict_stats

In [8]:
from tqdm import tqdm

dossier_games = 'games'
dossier_esports = 'esports-data'

with open("./esports-data/mapping_data.json", 'r') as mapping_file:
    mappings_data_list = json.load(mapping_file)

mappings_data = {item['platformGameId']: item for item in mappings_data_list}

# List only files with the `.json` extension without building intermediary lists
games = (f for f in os.listdir(dossier_games) if os.path.isfile(os.path.join(dossier_games, f)) and f.endswith('.json') )
list_game=[]

for game in tqdm(games):
    try:
        chemin_complet = os.path.join(dossier_games, game)
        with open(chemin_complet, 'r') as path_to_json_game:
            game_json = json.load(path_to_json_game)

        dict_game=final_data(game_json)
        list_game.append(dict_game)
    except KeyError :
        print(game)


game_dataFrame=pd.DataFrame(list_game)


602it [00:01, 504.99it/s]

ESPORTSTMNT03_3198743.json


754it [00:01, 422.67it/s]

ESPORTSTMNT04_2689609.json


1005it [00:02, 474.10it/s]

ESPORTSTMNT06_2819332.json
ESPORTSTMNT06_2821295.json
ESPORTSTMNT06_2823222.json


In [9]:
game_dataFrame

,gameDate,esportsPlatformId,gameVersion,NbCampsSecuredBlue,NbCampsSecuredRed,NbCampsStolenBlue,NbCampsStolenRed,NbScuttlesBlue,NbScuttlesRed,NbRiftHeraldsBlue,...,TotalCCDurationMidBlue,TotalCCDurationADBlue,TotalCCDurationSupBlue,TotalCCDurationTopRed,TotalCCDurationJgRed,TotalCCDurationMidRed,TotalCCDurationADRed,TotalCCDurationSupRed,winner,DragonSoulTeam
0,2023-06-08 22:13:11.713000+00:00,ESPORTSTMNT01:3363718,13.11.512.8126,38,37,0,1,2,7,0,...,25.155140,5.204233,7.129888,7.776570,8.207527,26.687611,7.391943,18.132963,red,NaN
1,2023-06-08 23:10:45.192000+00:00,ESPORTSTMNT01:3363735,13.11.512.8126,25,28,1,0,4,3,2,...,35.029465,11.287741,11.529725,3.611837,5.032828,9.233871,20.212564,10.568647,blue,NaN
2,2023-06-08 21:11:35.129000+00:00,ESPORTSTMNT01:3364704,13.11.512.8126,32,35,1,0,6,1,2,...,10.314743,13.506590,30.223419,16.036753,30.994061,32.933189,6.644125,20.557760,blue,NaN
3,2023-06-09 01:28:34.502000+00:00,ESPORTSTMNT01:3364786,13.11.512.8126,26,38,0,1,2,7,2,...,23.978035,14.803380,24.635223,27.651712,25.113615,24.579681,10.638018,50.089462,red,NaN
4,2023-06-09 00:06:49.799000+00:00,ESPORTSTMNT01:3365747,13.11.512.8126,37,47,1,1,8,3,2,...,18.393494,9.685623,5.784781,29.541517,141.538055,42.257111,13.964274,23.228823,blue,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-08-07 10:13:39.522000+00:00,LPL_A:343971,13.13.518.1870,30,39,1,0,5,4,0,...,4.121802,3.727788,34.811211,11.295624,18.743952,15.502586,11.846214,19.336792,blue,blue
996,2023-08-07 11:11:51.015000+00:00,LPL_A:343972,13.13.518.1870,33,28,2,0,5,3,1,...,5.148436,3.010280,28.095350,16.112463,11.018398,5.550389,12.581329,12.781189,blue,NaN
997,2023-08-08 09:21:12.797000+00:00,LPL_A:343990,13.13.518.1870,28,27,1,1,3,4,1,...,1.466858,3.164314,19.997421,33.827484,29.596952,3.144720,3.155783,34.589291,red,NaN
998,2023-08-08 10:10:59.294000+00:00,LPL_A:343995,13.13.518.1870,26,29,1,1,0,6,0,...,7.935424,12.953035,44.031033,22.251495,17.872515,10.131514,0.804042,37.521431,blue,NaN
